In [6]:
import model_conn_2_rev_2 as md

In [10]:
data = md.fetch_recent_rul_logs()
array_data = md.dict_to_array(data)
avg_data_0 = md.compute_moving_average(array_data[:,0],500)
avg_data_1 = md.compute_moving_average(array_data[:,1],500)
avg_data_2 = md.compute_moving_average(array_data[:,2],500)

In [17]:
data = (float(avg_data_0),float(avg_data_1),float(avg_data_2))
data

(615368.53308, 697524.90384, 9032713.236000001)

In [1]:
import model_conn_1_rev_2 as md

## rul test

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./for_test/01_M02_DC_train_pre.csv')
df

,Unnamed: 0,time,Tool,stage,Lot,runnum,recipe,recipe_step,IONGAUGEPRESSURE,ETCHBEAMVOLTAGE,...,ETCHGASCHANNEL1READBACK,ETCHPBNGASREADBACK,FIXTURETILTANGLE,ROTATIONSPEED,ACTUALROTATIONANGLE,FIXTURESHUTTERPOSITION,ETCHSOURCEUSAGE,ETCHAUXSOURCETIMER,ETCHAUX2SOURCETIMER,ACTUALSTEPDURATION
0,0,3283834,01M02,1,9253,10511673,67,1,-0.079428,-0.958814,...,-1.751661,-2.783162,2.027848,-0.013207,-0.122437,0.0,2.395370,2.871511,3.175941,4.037785
1,1,3283838,01M02,1,9253,10511673,67,1,-0.079419,-0.958928,...,-1.743905,-2.780884,2.027848,-0.013207,-0.122437,0.0,2.395370,2.871511,3.175941,4.037785
2,2,3283842,01M02,1,9253,10511673,67,1,-0.079409,-0.959156,...,-1.743905,-2.780884,2.027848,-0.013207,-0.122437,0.0,2.395370,2.871511,3.175941,4.037785
3,3,3283846,01M02,1,9253,10511673,67,1,-0.079420,-0.958928,...,-1.747158,-2.780884,2.027848,-0.013207,-0.122437,0.0,2.395370,2.871511,3.175941,4.037785
4,4,3283850,01M02,1,9253,10511673,67,1,-0.079430,-0.959269,...,-1.747158,-2.784389,2.027848,-0.013207,-0.122437,0.0,2.395370,2.871511,3.175941,4.037785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5092350,5092350,39789824,01M02,1,9978,12838177,67,27,0.026573,0.711354,...,1.410382,0.362004,0.022131,-0.013207,-0.122455,1.0,-0.184711,-0.021699,0.168088,-0.304990
5092351,5092351,39789828,01M02,1,9978,12838177,67,27,0.026573,0.710216,...,1.410382,0.359727,0.022131,-0.013207,-0.122455,1.0,-0.184707,-0.021695,0.168092,-0.304990
5092352,5092352,39789832,01M02,1,9978,12838177,67,27,0.026573,0.711012,...,1.416896,0.359727,0.022131,-0.013207,-0.122455,1.0,-0.184704,-0.021692,0.168096,-0.304990
5092353,5092353,39789836,01M02,1,9978,12838177,67,27,0.026573,0.711354,...,1.416896,0.359727,0.022131,-0.013207,-0.122455,1.0,-0.184701,-0.021688,0.168100,-0.304990


In [4]:
df = df.iloc[:,1:]


In [5]:


target_time = 3907068-3283834
df3 = df[(df['time'] >= 3283834 )&(df['time'] <= 3907068 )]
df3 = df3.head(target_time)
df3 = df3.reset_index(drop=True)
# Copy the last row of df3 to create additional rows
new_rows = 100  # Number of rows to add
for i in range(new_rows):
    # Copy the last row and adjust the 'time' column
    last_row = df3.iloc[-1].copy()
    last_row['time'] += 4
    
    # Append the modified row to the DataFrame using pd.concat
    df3 = pd.concat([df3, last_row.to_frame().T], ignore_index=True)

X_cols = ['time', 'stage', 'Lot', 'runnum', 'recipe', 'recipe_step',
       'IONGAUGEPRESSURE', 'ETCHBEAMVOLTAGE', 'ETCHBEAMCURRENT',
       'ETCHSUPPRESSORVOLTAGE', 'ETCHSUPPRESSORCURRENT', 'FLOWCOOLFLOWRATE',
       'FLOWCOOLPRESSURE', 'ETCHGASCHANNEL1READBACK', 'ETCHPBNGASREADBACK',
       'FIXTURETILTANGLE', 'ROTATIONSPEED', 'ACTUALROTATIONANGLE',
       'FIXTURESHUTTERPOSITION', 'ETCHSOURCEUSAGE', 'ETCHAUXSOURCETIMER',
       'ETCHAUX2SOURCETIMER', 'ACTUALSTEPDURATION']

df3_target = df3[X_cols]
dict_format = df3_target.to_dict(orient='records')
result = md.predict_with_xgb_model_optimized(dict_format,length=target_time)
result

1/1 [==============================] - 0s 38ms/step


KeyboardInterrupt: 

In [34]:
def remove_outliers_with_moving_window(data, window_size):
    """
    이동 구간을 사용하여 데이터의 이상치를 제거합니다.

    :param data: 이상치를 제거할 데이터 리스트
    :param window_size: 이동 구간의 크기
    :return: 이상치가 제거된 데이터 리스트
    """
    # 결과를 저장할 리스트
    cleaned_data = []

    # 이동 구간을 통해 데이터를 반복합니다.
    for i in range(0, len(data) - window_size + 1):
        window_data = data[i:i+window_size]

        # Q1, Q3 및 IQR 계산
        Q1 = np.percentile(window_data, 25)
        Q3 = np.percentile(window_data, 75)
        IQR = Q3 - Q1

        # 이동 구간의 중앙값이 이상치인지 확인하고 cleaned_data에 추가합니다.
        median = window_data[window_size // 2]
        if Q1 - 1.5 * IQR <= median <= Q3 + 1.5 * IQR:
            cleaned_data.append(median)

    # 마지막 window_size - 1개의 데이터는 처리되지 않았으므로 원래 데이터에서 추가합니다.
    cleaned_data.extend(data[-(window_size - 1):])

    return cleaned_data

In [35]:
import numpy as np

In [36]:
outlier_removed = remove_outliers_with_moving_window(np.array(result)[:,2],500)
outlier_removed

[2687.981,
 4922.8975,
 3771.875,
 3236.4126,
 4426.364,
 4409.041,
 6587.3027,
 5261.134,
 2445.231,
 5882.891,
 6359.604,
 8943.283,
 7786.242,
 7427.551,
 6916.1953,
 7883.4346,
 7662.5815,
 8479.101,
 8465.619,
 8438.303,
 9277.256,
 10151.136,
 10039.2705,
 10052.166,
 9838.41,
 10089.802,
 9627.611,
 10328.426,
 10369.698,
 10498.683,
 10505.564,
 11218.659,
 11031.409,
 10939.819,
 11063.613,
 10911.199,
 10575.592,
 10359.869,
 10154.962,
 10130.922,
 10025.813,
 4989.3604,
 4791.774,
 4684.261,
 5093.6978,
 4622.353,
 5240.9004,
 4523.496,
 4508.2983,
 4381.669,
 4080.7996,
 3782.0754,
 3772.606,
 4062.797,
 -365.29327,
 -534.94855,
 -266.8426,
 -1104.3601,
 -1247.8035,
 -1536.3508,
 -1684.7131,
 -1712.846,
 -3113.5488,
 -3185.73,
 -3189.1091,
 -3510.3232,
 -3798.8801,
 -3762.4583,
 -4135.653,
 -4172.018,
 -4406.9253,
 -4367.611,
 -4369.576,
 -4395.082,
 -4417.5176,
 -4581.117,
 -4617.444,
 -4644.881,
 -4668.239,
 -4705.5376,
 -4843.1875,
 -4836.314,
 -4834.5684,
 -4840.902,
 

In [37]:
moving_avg = md.compute_moving_average(outlier_removed,500)
moving_avg

array([2.63012282e+02, 2.58088625e+02, 2.48696550e+02, ...,
       7.15580701e+05, 7.15518919e+05, 7.15457140e+05])

In [39]:
list(moving_avg)

[263.012282409668,
 258.08862460327146,
 248.69655044555662,
 241.60985162353515,
 235.60765734863278,
 227.20459338378905,
 218.8444419250488,
 206.13967440795898,
 196.09955169677735,
 191.68741351318363,
 180.3915927124024,
 168.1396854858399,
 150.7233772888184,
 135.63897625732423,
 121.27429119873045,
 107.91511248779297,
 92.62511727905272,
 77.79774716186522,
 61.34178124999998,
 44.916270416259735,
 28.53577178955076,
 10.46348654174804,
 -9.355411071777361,
 -28.960781127929703,
 -48.57507028198243,
 -67.75692828369142,
 -87.43476025390626,
 -106.18125292968753,
 -126.34556982421877,
 -146.60868423461912,
 -167.12511401367186,
 -187.67031518554688,
 -209.62599057006835,
 -231.18412835693363,
 -252.54766558837895,
 -274.1705885314941,
 -295.4911582946777,
 -316.1214327087402,
 -336.32984725952144,
 -356.14113793945313,
 -375.9076652832031,
 -395.4601946716308,
 -404.9389455871582,
 -414.02524890136715,
 -422.88722018432617,
 -432.56635787963864,
 -441.29068417358394,
 -451.252

In [53]:

# df = pd.read_csv('./for_test/M04_dummy.csv')
# df

In [46]:
# Copy the last row of df3 to create additional rows
new_rows = 100  # Number of rows to add
for i in range(new_rows):
    # Copy the last row and adjust the 'time' column
    last_row = df3.iloc[-1].copy()
    last_row['time'] += 4
    
    # Append the modified row to the DataFrame using pd.concat
    df3 = pd.concat([df3, last_row.to_frame().T], ignore_index=True)

X_cols = ['time', 'stage', 'Lot', 'runnum', 'recipe', 'recipe_step',
       'IONGAUGEPRESSURE', 'ETCHBEAMVOLTAGE', 'ETCHBEAMCURRENT',
       'ETCHSUPPRESSORVOLTAGE', 'ETCHSUPPRESSORCURRENT', 'FLOWCOOLFLOWRATE',
       'FLOWCOOLPRESSURE', 'ETCHGASCHANNEL1READBACK', 'ETCHPBNGASREADBACK',
       'FIXTURETILTANGLE', 'ROTATIONSPEED', 'ACTUALROTATIONANGLE',
       'FIXTURESHUTTERPOSITION', 'ETCHSOURCEUSAGE', 'ETCHAUXSOURCETIMER',
       'ETCHAUX2SOURCETIMER', 'ACTUALSTEPDURATION']

df3_target = df3[X_cols]
dict_format = df3_target.to_dict(orient='records')
result = md.predict_with_xgb_model_optimized(dict_format,length=25)
result

1/1 [==============================] - 0s 20ms/step


[[2866532.5, 591067.06, 49256.406],
 [2866571.2, 590884.75, 49253.4],
 [2866507.8, 591008.9, 49223.277],
 [2866483.5, 590896.7, 49226.734],
 [2866438.8, 590881.56, 49219.754],
 [2866448.5, 590861.6, 49220.254],
 [2866464.5, 590813.2, 49217.49],
 [2866409.8, 590838.56, 49188.703],
 [2866386.8, 590902.06, 49190.33],
 [2866419.0, 590960.56, 49186.14],
 [2866416.0, 590948.25, 49185.375],
 [2866444.0, 590818.94, 49188.33],
 [2866338.5, 590961.2, 49160.668],
 [2866354.5, 590796.1, 49159.516],
 [2866393.5, 590588.3, 49166.492],
 [2866400.5, 590536.56, 49167.055],
 [2866401.0, 590577.7, 49169.152],
 [2866223.0, 590695.0, 49150.023],
 [2866256.8, 590774.0, 49139.812],
 [2866232.5, 590745.06, 49136.0],
 [2866224.8, 590806.0, 49136.023],
 [2866232.5, 590787.75, 49137.2],
 [2866214.0, 590823.94, 49134.703],
 [2866050.8, 590754.44, 49118.633],
 [2866050.8, 590754.44, 49118.633]]

In [47]:
X_cols = ['time', 'stage', 'Lot', 'runnum', 'recipe', 'recipe_step',
       'IONGAUGEPRESSURE', 'ETCHBEAMVOLTAGE', 'ETCHBEAMCURRENT',
       'ETCHSUPPRESSORVOLTAGE', 'ETCHSUPPRESSORCURRENT', 'FLOWCOOLFLOWRATE',
       'FLOWCOOLPRESSURE', 'ETCHGASCHANNEL1READBACK', 'ETCHPBNGASREADBACK',
       'FIXTURETILTANGLE', 'ROTATIONSPEED', 'ACTUALROTATIONANGLE',
       'FIXTURESHUTTERPOSITION', 'ETCHSOURCEUSAGE', 'ETCHAUXSOURCETIMER',
       'ETCHAUX2SOURCETIMER', 'ACTUALSTEPDURATION']

In [48]:
df3_target = df3[X_cols]
df3_target

,time,stage,Lot,runnum,recipe,recipe_step,IONGAUGEPRESSURE,ETCHBEAMVOLTAGE,ETCHBEAMCURRENT,ETCHSUPPRESSORVOLTAGE,...,ETCHGASCHANNEL1READBACK,ETCHPBNGASREADBACK,FIXTURETILTANGLE,ROTATIONSPEED,ACTUALROTATIONANGLE,FIXTURESHUTTERPOSITION,ETCHSOURCEUSAGE,ETCHAUXSOURCETIMER,ETCHAUX2SOURCETIMER,ACTUALSTEPDURATION
0,22075412,1,9349,11744123,66,5,-0.079146,-0.958758,-0.959297,-0.946148,...,-1.75041,-2.788945,-0.200727,-0.013207,0.379009,0.0,-1.471444,-1.454852,-1.321854,-0.291949
1,22075416,1,9349,11744123,66,5,-0.079146,-0.959042,-0.959935,-0.94624,...,-1.75041,-2.789997,-0.200727,-0.013207,0.379009,0.0,-1.471444,-1.454852,-1.321854,-0.291949
2,22075420,1,9349,11744123,66,5,-0.079146,-0.958644,-0.959722,-0.946469,...,-1.747158,-2.789997,-0.200727,-0.013207,0.379009,0.0,-1.471444,-1.454852,-1.321854,-0.291949
3,22075424,1,9349,11744123,66,5,-0.079146,-0.959099,-0.959297,-0.946103,...,-1.747158,-2.789997,-0.200727,-0.013207,0.379009,0.0,-1.471444,-1.454852,-1.321854,-0.291949
4,22075428,1,9349,11744123,66,5,-0.079146,-0.958814,-0.959722,-0.946424,...,-1.756665,-2.788945,-0.200727,-0.013207,0.379009,0.0,-1.471444,-1.454852,-1.321854,-0.291949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,22075892,1,9349,11744123,66,5,-0.079146,-0.958871,-0.959864,-0.946057,...,-1.753413,-2.788945,-0.200727,-0.013207,0.379009,0.0,-1.471444,-1.454852,-1.321854,-0.291949
121,22075896,1,9349,11744123,66,5,-0.079146,-0.958871,-0.959864,-0.946057,...,-1.753413,-2.788945,-0.200727,-0.013207,0.379009,0.0,-1.471444,-1.454852,-1.321854,-0.291949
122,22075900,1,9349,11744123,66,5,-0.079146,-0.958871,-0.959864,-0.946057,...,-1.753413,-2.788945,-0.200727,-0.013207,0.379009,0.0,-1.471444,-1.454852,-1.321854,-0.291949
123,22075904,1,9349,11744123,66,5,-0.079146,-0.958871,-0.959864,-0.946057,...,-1.753413,-2.788945,-0.200727,-0.013207,0.379009,0.0,-1.471444,-1.454852,-1.321854,-0.291949


In [49]:
dict_format = df3_target.to_dict(orient='records')
dict_format

[{'time': 22075412,
  'stage': 1,
  'Lot': 9349,
  'runnum': 11744123,
  'recipe': 66,
  'recipe_step': 5,
  'IONGAUGEPRESSURE': -0.0791455370139089,
  'ETCHBEAMVOLTAGE': -0.9587575645021268,
  'ETCHBEAMCURRENT': -0.9592965888104408,
  'ETCHSUPPRESSORVOLTAGE': -0.946148462426951,
  'ETCHSUPPRESSORCURRENT': -0.9390426794102577,
  'FLOWCOOLFLOWRATE': -2.841849516088073,
  'FLOWCOOLPRESSURE': 0.8427449270435082,
  'ETCHGASCHANNEL1READBACK': -1.7504101432154815,
  'ETCHPBNGASREADBACK': -2.788945073824961,
  'FIXTURETILTANGLE': -0.2007267092824509,
  'ROTATIONSPEED': -0.0132071874464973,
  'ACTUALROTATIONANGLE': 0.3790085681863171,
  'FIXTURESHUTTERPOSITION': 0.0,
  'ETCHSOURCEUSAGE': -1.4714444846113746,
  'ETCHAUXSOURCETIMER': -1.4548524651976775,
  'ETCHAUX2SOURCETIMER': -1.321853532275452,
  'ACTUALSTEPDURATION': -0.2919485995520355},
 {'time': 22075416,
  'stage': 1,
  'Lot': 9349,
  'runnum': 11744123,
  'recipe': 66,
  'recipe_step': 5,
  'IONGAUGEPRESSURE': -0.0791455370139089,
  'E

In [50]:
result = md.predict_with_xgb_model_optimized(dict_format,length=25)

1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 21ms/step


In [62]:
result

[[2905193.2, 729261.56, 2078.2625],
 [2904995.0, 726337.75, 2632.836],
 [2892175.8, 726470.94, 2687.981],
 [2908951.0, 730066.25, 4922.8975],
 [2916742.0, 730922.5, 3771.875],
 [2897638.2, 730940.94, 3236.4126],
 [2905183.0, 735816.1, 4426.364],
 [2900390.5, 734718.6, 4409.041],
 [2860430.5, 730398.1, 6587.3027],
 [2858038.2, 743042.7, 5261.134],
 [2846803.8, 746939.44, 2445.231],
 [2850805.2, 740033.1, 5882.891],
 [2878982.8, 736631.8, 6359.604],
 [2878344.2, 729571.1, 8943.283],
 [2880265.0, 738378.6, 7786.242],
 [2886647.2, 730991.75, 7427.551],
 [2901937.8, 733046.94, 6916.1953],
 [2883264.0, 713199.25, 7883.4346],
 [2872202.0, 699444.44, 7662.5815],
 [2893355.5, 673687.56, 8479.101],
 [2895130.0, 671844.94, 8465.619],
 [2905496.5, 668460.0, 8438.303],
 [2904301.8, 657871.25, 9277.256],
 [2914363.2, 654093.8, 10151.136],
 [2935920.0, 637730.44, 10039.2705],
 [2947089.2, 632527.2, 10052.166],
 [2937966.5, 627387.6, 9838.41],
 [2961339.8, 625500.7, 10089.802],
 [2955913.0, 627101.8, 

In [74]:
import numpy as np
np.array(result)

array([2.9051932e+06, 7.2926156e+05, 2.0782625e+03], dtype=float32)

TypeError: list indices must be integers or slices, not tuple

In [77]:
md.remove_outliers_with_moving_window(result,50)

AttributeError: module 'model_conn_1_rev_2' has no attribute 'remove_outliers_with_moving_window'

In [76]:
md.compute_moving_average(np.array(result)[:,2],50)

array([ 7.55679731e+03,  7.60539803e+03,  7.64037469e+03,  7.66823106e+03,
        7.64541462e+03,  7.64542924e+03,  7.66195693e+03,  7.56612379e+03,
        7.46724400e+03,  7.33016109e+03,  7.20285121e+03,  7.12899052e+03,
        6.98060568e+03,  6.81971934e+03,  6.60659676e+03,  6.38860094e+03,
        6.17633532e+03,  5.97422923e+03,  5.74635408e+03,  5.51712484e+03,
        5.27229367e+03,  5.02026823e+03,  4.76806181e+03,  4.49437819e+03,
        4.20400326e+03,  3.91582632e+03,  3.62688136e+03,  3.34176281e+03,
        3.04834443e+03,  2.76344333e+03,  2.46397719e+03,  2.16321845e+03,
        1.85913405e+03,  1.55215901e+03,  1.23105955e+03,  9.13739996e+02,
        5.98125573e+02,  2.80472448e+02, -3.40950037e+01, -3.40548715e+02,
       -6.42296107e+02, -9.39259398e+02, -1.23616084e+03, -1.53104414e+03,
       -1.72527653e+03, -1.91529163e+03, -2.10376224e+03, -2.30022232e+03,
       -2.48637291e+03, -2.66800613e+03, -2.83163265e+03, -2.99158619e+03,
       -3.14747607e+03, -

In [68]:
import main
bar_lis, line_lis = main.fetch_bar_lis_from_database()
print(len(bar_lis))
bar_lis


86


[[datetime.datetime(2023, 10, 29, 15, 57, 10), 1877, 1877],
 [datetime.datetime(2023, 10, 29, 15, 54, 54), 1843, 1843],
 [datetime.datetime(2023, 10, 29, 15, 54, 5), 1831, 1831],
 [datetime.datetime(2023, 10, 29, 15, 53, 9), 1817, 1817],
 [datetime.datetime(2023, 10, 29, 15, 52, 13), 1803, 1803],
 [datetime.datetime(2023, 10, 29, 15, 51, 45), 1796, 1796],
 [datetime.datetime(2023, 10, 29, 15, 51, 17), 1789, 1789],
 [datetime.datetime(2023, 10, 29, 15, 49, 45), 1766, 1766],
 [datetime.datetime(2023, 10, 29, 15, 48, 33), 1748, 1748],
 [datetime.datetime(2023, 10, 29, 15, 48, 13), 1743, 1743],
 [datetime.datetime(2023, 10, 29, 15, 46, 12), 1713, 1713],
 [datetime.datetime(2023, 10, 29, 15, 42, 28), 1657, 1619],
 [datetime.datetime(2023, 10, 29, 15, 39, 52), 1618, 1618],
 [datetime.datetime(2023, 10, 29, 15, 39, 3), 1606, 1606],
 [datetime.datetime(2023, 10, 29, 15, 37, 23), 1581, 1581],
 [datetime.datetime(2023, 10, 29, 15, 34, 23), 1536, 1536],
 [datetime.datetime(2023, 10, 29, 15, 33, 7

In [ ]:
bar_lis

In [2]:
line_lis

[78,
 174,
 175,
 243,
 244,
 355,
 356,
 460,
 461,
 462,
 463,
 464,
 465,
 466,
 467,
 468,
 469,
 561,
 562,
 563,
 564,
 580,
 581,
 582,
 583,
 696,
 697,
 760,
 761,
 762,
 763,
 781,
 782,
 783,
 788,
 789,
 790,
 791,
 792,
 871,
 872,
 873,
 876,
 877,
 976,
 977,
 978,
 994,
 995,
 996]

In [1]:
abnormal_models_path = ['./model/abnormal_detect/RF_model(FL).pkl','./model/abnormal_detect/RF_model(PB).pkl','./model/abnormal_detect/RF_model(PH).pkl']
rul_models_path = ['./model/RULmodel/231023_xgb_ss_df_fl.pickle','./model/RULmodel/231023_xgb_ss_df_pb.pickle','./model/RULmodel/231023_xgb_ss_df_fl.pickle']
inverse_scalers_path = ['./model/RULmodel/231023_ss_y_df_fl.pickle','./model/RULmodel/231023_ss_y_df_pb.pickle','./model/RULmodel/231023_ss_y_df_ph.pickle']
scalers_path = ['./model/RULmodel/231023_ss_x_df_fl.pickle','./model/RULmodel/231023_ss_x_df_pb.pickle','./model/RULmodel/231023_ss_x_df_ph.pickle']

In [2]:
import joblib

def load_object_with_joblib(path):
    return joblib.load(path)

def get_model_features_count(model):
    try:
        return len(model.feature_importances_)
    except AttributeError:
        return "Not available for this model"

def get_scaler_feature_count(scaler):
    try:
        return scaler.n_features_in_
    except AttributeError:
        return "Not available for this scaler"

# 모델의 컬럼 개수 확인
for path in abnormal_models_path:
    model = load_object_with_joblib(path)
    print(f"{path} model features count: {get_model_features_count(model)}")

# 스케일러의 컬럼 개수 확인
for path in scalers_path:
    scaler = load_object_with_joblib(path)
    print(f"{path} scaler features count: {get_scaler_feature_count(scaler)}")


./model/abnormal_detect/RF_model(FL).pkl model features count: 10
./model/abnormal_detect/RF_model(PB).pkl model features count: 10
./model/abnormal_detect/RF_model(PH).pkl model features count: 10
./model/RULmodel/231023_ss_x_df_fl.pickle scaler features count: 10
./model/RULmodel/231023_ss_x_df_pb.pickle scaler features count: 10
./model/RULmodel/231023_ss_x_df_ph.pickle scaler features count: 9


In [84]:
import pymysql
import pickle
import xgboost as xgb
import numpy as np
import pandas as pd
import time
import warnings
from datetime import datetime
import joblib

# XGBoost 관련 경고 숨기기
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

# 현재 시간 가져오기
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

## 길이 기준 설정
avg_len = 500

def fetch_recent_logs(length=avg_len):
    """mysql db에서 최근 로그를 가져오는 함수"""
    # mysql 데이터베이스에 연결

    connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            # 가장 최근의 데이터부터 지정한 길이만큼 가져오는 SQL 쿼리
            sql = f'''SELECT IONGAUGEPRESSURE, ETCHBEAMVOLTAGE, ETCHBEAMCURRENT, ETCHSUPPRESSORVOLTAGE, ETCHSUPPRESSORCURRENT,
            FLOWCOOLFLOWRATE, FLOWCOOLPRESSURE, ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
            FIXTURETILTANGLE, ROTATIONSPEED, ACTUALROTATIONANGLE,
            ETCHSOURCEUSAGE, ETCHAUXSOURCETIMER, ETCHAUX2SOURCETIMER,
            ACTUALSTEPDURATION FROM input_data_1 ORDER BY input_time DESC LIMIT {length}'''
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        connection.close()

    return results

def fetch_recent_logs_for_multi(length=avg_len):
    """mysql db에서 최근 로그를 가져오는 함수"""
    # mysql 데이터베이스에 연결

    connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            # 가장 최근의 데이터부터 지정한 길이만큼 가져오는 SQL 쿼리
            sql = f'''SELECT ACTUALROTATIONANGLE, ACTUALSTEPDURATION, ETCHBEAMCURRENT, ETCHGASCHANNEL1READBACK, 
              ETCHPBNGASREADBACK, ETCHSOURCEUSAGE, FIXTURETILTANGLE, FLOWCOOLFLOWRATE, FLOWCOOLPRESSURE, 
              IONGAUGEPRESSURE FROM input_data_1 ORDER BY input_time DESC LIMIT {length}'''
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        connection.close()

    return results

def dict_to_array(data):
    """사전 형태의 데이터를 2차원 넘파이 배열로 변환하는 함수"""
    return np.array([list(item.values()) for item in data])


def predict_with_xgb_model(data,model_path,inverse_scaler_path):
    """xgboost 모델을 사용해 예측하는 함수"""
    # 데이터 형태 변환
    transformed_data = dict_to_array(data)

    """모델 예측 전 전처리 함수"""
    # 모델 불러오기
    scaler = joblib.load('./model/rul/X_scaler.pkl')
    
    # scaler 적용
    scaled_data = scaler.transform(transformed_data)

    """xgboost 모델을 사용해 예측하는 함수"""
    # 모델 불러오기
    model =  joblib.load(model_path)

    # 예측 실행
    predictions_scaled = model.predict(scaled_data)

    #### rul inverse transfomr 적용 필요!!!!
     # 모델 불러오기
    inverse_scaler = joblib.load(inverse_scaler_path)
    
    predictions = inverse_scaler.inverse_transform(predictions_scaled.reshape(-1,1))

    return predictions[:,0]

def predict_with_xgb_multi_model(data,model_path):
    """xgboost 다중분류 모델을 사용해 예측하는 함수"""
    # 데이터 형태 변환
    transformed_data = dict_to_array(data)

    """모델 예측 전 전처리 함수"""
    # 모델 불러오기
    scaler = joblib.load('./model/abnormal_detect/StandardScaler.pkl')
    
    # scaler 적용
    scaled_data = scaler.transform(transformed_data)

    """xgboost 모델을 사용해 예측하는 함수"""
    # 모델 불러오기
    model=  joblib.load(model_path) 

    # 예측 실행
    predictions = model.predict(scaled_data)

    return predictions

def compute_moving_average(data, window_size=avg_len):
    """이동평균 계산하는 함수"""
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

import numpy as np

def compute_moving_median(data, window_size):
    """이동 중앙값 계산하는 함수"""
    tmp_data = data.reshape(1,-1)
    num_data = len(tmp_data)
    medians = []

    for i in range(num_data - window_size + 1):
        window_data = tmp_data[i:i+window_size]
        medians.append(np.median(window_data))

    return np.array(medians)

########################################################################################################################################
########################################################################################################################################
########################################################################################################################################

### 데이터 밀어 넣기

def insert_single_data(connection, single_data):
    try:
        with connection.cursor() as cursor:
            # 현재 시간 가져오기
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO input_data_1 (time, Tool, stage, Lot, runnum, recipe, recipe_step,
       IONGAUGEPRESSURE, ETCHBEAMVOLTAGE, ETCHBEAMCURRENT,
       ETCHSUPPRESSORVOLTAGE, ETCHSUPPRESSORCURRENT, FLOWCOOLFLOWRATE,
       FLOWCOOLPRESSURE, ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
       FIXTURETILTANGLE, ROTATIONSPEED, ACTUALROTATIONANGLE,
       FIXTURESHUTTERPOSITION, ETCHSOURCEUSAGE, ETCHAUXSOURCETIMER,
       ETCHAUX2SOURCETIMER, ACTUALSTEPDURATION, input_time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, "{current_time}")'''
            cursor.execute(sql, single_data)
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()
    return current_time

def insert_single_rul_data(connection, data, current_time):
    try:
        with connection.cursor() as cursor:
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO rul_1(rul_fl, rul_pb, rul_ph, input_time) 
                      VALUES (%s, %s, %s, "{current_time}")'''
            cursor.execute(sql, (data[0], data[1], data[2]))
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()

def insert_single_multi_data(connection, data, current_time):
    try:
        with connection.cursor() as cursor:
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO multi_1(multi_pred_fl, multi_pred_pb, multi_pred_ph, input_time) 
                      VALUES (%s, %s, %s, "{current_time}")'''
            cursor.execute(sql, (data[0], data[1], data[2]))
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()




def main():
    # 데이터베이스 연결 설정

    connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    
    # CSV 파일 읽기
    df = pd.read_csv("./model_data_input/test.csv")

    # DataFrame에서 튜플 리스트로 데이터 변환
    data_tuples = list(df.itertuples(index=False, name=None))

    abnormal_models_path = ['./model/abnormal_detect/RF_model(FL).pkl','./model/abnormal_detect/RF_model(PB).pkl','./model/abnormal_detect/RF_model(PH).pkl']
    rul_models_path = ['./model/rul/xgb_model_for_fl.pkl','./model/rul/xgb_model_for_pb.pkl','./model/rul/xgb_model_for_ph.pkl']
    inverse_scalers_path = ['./model/rul/y_scaler_for_fl.pkl','./model/rul/y_scaler_for_pb.pkl','./model/rul/y_scaler_for_ph.pkl']

    ## 데이터를 한줄 씩 밀어넣으면서 진행하는 방식

    for single_data in data_tuples:

        try:
            # 데이터 삽입
            current_time = insert_single_data(connection, single_data)

            data = fetch_recent_logs(length=avg_len)
            data_for_multi = fetch_recent_logs_for_multi(length=avg_len)

            rul_insert = []
            multi_insert = []

            for i in range(len(abnormal_models_path)):
            
                # 가져온 데이터를 기반으로 예측하기
                predictions = predict_with_xgb_model(data,rul_models_path[i],inverse_scalers_path[i])
                predictions_for_multi = predict_with_xgb_multi_model(data_for_multi,abnormal_models_path[i])
                
                # 이동평균 계산하기
                window_size = min(avg_len, len(predictions))  # 데이터 수와 avg_len 중 작은 값을 창 크기로 사용
                moving_avg = compute_moving_average(predictions, window_size=window_size)
                window_size_for_multi = min(avg_len, len(predictions_for_multi))  # 데이터 수와 avg_len 중 작은 값을 창 크기로 사용
                moving_avg_for_multi = compute_moving_average(predictions_for_multi, window_size=window_size_for_multi)
                
                rul_insert.append(moving_avg[0])
                print(6)
                multi_insert.append(predictions_for_multi[0])
                print(7)
                
            # pred 데이터 삽입
            insert_single_rul_data(connection, rul_insert, current_time)
            print(8)
            insert_single_multi_data(connection, multi_insert, current_time)
            print(9)

            time.sleep(4)  # 4초 대기

        except Exception as e:
            print(f"Error: {e}")
    
    # 연결 종료
    connection.close()    

main()


c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


6
7
6
7
6
7
8
9


c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


6
7
6
7
6
7
8
9


c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


6
7
6
7
6
7
8
9


KeyboardInterrupt: 

In [75]:

# CSV 파일 읽기
df = pd.read_csv("./model_data_input/test.csv")

# DataFrame에서 튜플 리스트로 데이터 변환
data_tuples = list(df.itertuples(index=False, name=None))

abnormal_models_path = ['./model/abnormal_detect/RF_model(FL).pkl','./model/abnormal_detect/RF_model(PB).pkl','./model/abnormal_detect/RF_model(PH).pkl']
rul_models_path = ['./model/rul/xgb_model_for_fl.pkl','./model/rul/xgb_model_for_pb.pkl','./model/rul/xgb_model_for_ph.pkl']
inverse_scalers_path = ['./model/rul/y_scaler_for_fl.pkl','./model/rul/y_scaler_for_pb.pkl','./model/rul/y_scaler_for_ph.pkl']

## 데이터를 한줄 씩 밀어넣으면서 진행하는 방식

In [11]:
connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)

NameError: name 'pymysql' is not defined

In [12]:
data = fetch_recent_logs(length=avg_len)
data

NameError: name 'fetch_recent_logs' is not defined

In [78]:
predictions = predict_with_xgb_model(data,rul_models_path[0],inverse_scalers_path[0])
predictions

c:\Users\ysj_1\miniconda3\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[2169449. ],
       [2169449. ],
       [2169449. ],
       [2169449. ],
       [1580659.8],
       [1541943.1],
       [1875674.9],
       [1819576.9],
       [1396611.4],
       [1735063.5],
       [1735063.5],
       [1733304.1],
       [2054692.5],
       [2056451.9],
       [1735063.5],
       [1733304.1],
       [1733304.1],
       [1733304.1],
       [1733304.1],
       [1733304.1],
       [1875674.9],
       [1733304.1],
       [2054692.5],
       [1497973.1],
       [1258518.9],
       [1344791.4],
       [1734558.6],
       [1489876.6],
       [1624971.2],
       [1562869.8],
       [1467559.1],
       [1693362.1],
       [1693362.1],
       [1692258.5],
       [1808169.8],
       [1861418.4],
       [1692258.5],
       [1692258.5],
       [1861418.4],
       [1861418.4],
       [1808169.8],
       [1778530.9],
       [1894442.5],
       [1894442.5],
       [1809929.1],
       [1694017.9],
       [1723802.4],
       [1839713.6],
       [1839713.6],
       [1892962.2],


In [79]:
window_size = min(avg_len, len(predictions))  # 데이터 수와 avg_len 중 작은 값을 창 크기로 사용
window_size

398

In [80]:
predictions[:,0]

array([2169449. , 2169449. , 2169449. , 2169449. , 1580659.8, 1541943.1,
       1875674.9, 1819576.9, 1396611.4, 1735063.5, 1735063.5, 1733304.1,
       2054692.5, 2056451.9, 1735063.5, 1733304.1, 1733304.1, 1733304.1,
       1733304.1, 1733304.1, 1875674.9, 1733304.1, 2054692.5, 1497973.1,
       1258518.9, 1344791.4, 1734558.6, 1489876.6, 1624971.2, 1562869.8,
       1467559.1, 1693362.1, 1693362.1, 1692258.5, 1808169.8, 1861418.4,
       1692258.5, 1692258.5, 1861418.4, 1861418.4, 1808169.8, 1778530.9,
       1894442.5, 1894442.5, 1809929.1, 1694017.9, 1723802.4, 1839713.6,
       1839713.6, 1892962.2, 1597665. , 1809259.9, 1978703. , 1936332. ,
       1978703. , 1978703. , 1995485.5, 1995485.5, 1775048.4, 1978703. ,
       1978703. , 1978703. , 1978703. , 1978703. , 2114300.5, 2049412.1,
       1978703. , 1978703. , 2114300.5, 1910645.6, 2114300.5, 1936332. ,
       1954776. , 2116059.8, 2116059.8, 1980462.4, 1980462.4, 1950332.5,
       1950332.5, 1927738.2, 1760793.6, 1469662.6, 

In [81]:
moving_avg = compute_moving_average(predictions[:,0], window_size=window_size)
moving_avg

array([1968009.60081658])

In [26]:
current_time = insert_single_data(connection, data_tuples[0])
current_time

'2023-10-24 12:36:08'

In [27]:
data_for_multi = fetch_recent_logs_for_multi(length=avg_len)
data_for_multi

[{'ACTUALROTATIONANGLE': -0.1224370708389037,
  'ACTUALSTEPDURATION': 4.03778468798202,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  'ETCHGASCHANNEL1READBACK': -1.751661148558742,
  'ETCHPBNGASREADBACK': -2.783162253978537,
  'ETCHSOURCEUSAGE': 2.3953704185454097,
  'FIXTURETILTANGLE': 2.027847975076401,
  'FLOWCOOLFLOWRATE': -2.842906518897486,
  'FLOWCOOLPRESSURE': -2.3328068957350747,
  'IONGAUGEPRESSURE': -0.0794283467426016},
 {'ACTUALROTATIONANGLE': -0.1224370708389037,
  'ACTUALSTEPDURATION': 4.03778468798202,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  'ETCHGASCHANNEL1READBACK': -1.751661148558742,
  'ETCHPBNGASREADBACK': -2.783162253978537,
  'ETCHSOURCEUSAGE': 2.3953704185454097,
  'FIXTURETILTANGLE': 2.027847975076401,
  'FLOWCOOLFLOWRATE': -2.842906518897486,
  'FLOWCOOLPRESSURE': -2.3328068957350747,
  'IONGAUGEPRESSURE': -0.0794283467426016},
 {'ACTUALROTATIONANGLE': -0.1224370708389037,
  'ACTUALSTEPDURATION': 4.03778468798202,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  

In [28]:
predictions_for_multi = predict_with_xgb_multi_model(data_for_multi,abnormal_models_path[0])
predictions_for_multi

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [29]:
multi_insert = []

In [30]:
multi_insert.append(predictions_for_multi[0])

In [31]:
multi_insert

[0]

In [17]:
len(data_for_multi[0].values())

10

In [19]:
import mysql.connector

In [20]:
db = mysql.connector.connect(
    host="limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com",
    user="oneday",
    password="1234",
    database="j6database",
)


# 커서 생성
cursor = db.cursor()


In [21]:
cursor.execute(f"""SELECT temp.*, (row_index - LAG(row_index) OVER (ORDER BY row_index)) as diff
                FROM (SELECT multi_pred_fl, multi_pred_pb, multi_pred_ph, input_time, ROW_NUMBER() OVER (ORDER BY input_time) AS row_index
                    FROM multi_1) AS temp
                WHERE (multi_pred_fl = 1) or (multi_pred_pb = 1) or (multi_pred_ph = 1);""")
existing_user = cursor.fetchall()
existing_user

[(0, 0, 1, datetime.datetime(2023, 10, 27, 23, 1, 43), 14, None),
 (1, 0, 1, datetime.datetime(2023, 10, 27, 23, 3, 23), 39, 25),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 4, 19), 53, 14),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 4, 23), 54, 1),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 5, 47), 75, 21),
 (1, 0, 1, datetime.datetime(2023, 10, 27, 23, 7, 52), 106, 31),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 9, 36), 132, 26),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 11, 20), 158, 26),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 11, 44), 164, 6),
 (1, 1, 0, datetime.datetime(2023, 10, 27, 23, 12, 20), 173, 9),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 14, 36), 207, 34),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 17, 1), 243, 36),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 18, 49), 270, 27),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 20, 33), 296, 26),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 21, 21), 308, 12),
 (0, 1, 0, datetime.date

In [42]:
(existing_user[2][-1]) != 1

True

In [23]:
bar_lis = [[existing_user[0][3], existing_user[0][4]]]
bar_lis

[[datetime.datetime(2023, 10, 27, 23, 1, 43), 14]]

In [24]:
for i in range(1, len(existing_user)):
    if existing_user[i][-1] != 1 :
        bar_lis[len(bar_lis)-1].append(existing_user[i-1][4])
        bar_lis.append([existing_user[i][3], existing_user[i][4]])
bar_lis[-1].append(existing_user[-1][4])

In [25]:
bar_lis

[[datetime.datetime(2023, 10, 27, 23, 1, 43), 14, 14],
 [datetime.datetime(2023, 10, 27, 23, 3, 23), 39, 39],
 [datetime.datetime(2023, 10, 27, 23, 4, 19), 53, 54],
 [datetime.datetime(2023, 10, 27, 23, 5, 47), 75, 75],
 [datetime.datetime(2023, 10, 27, 23, 7, 52), 106, 106],
 [datetime.datetime(2023, 10, 27, 23, 9, 36), 132, 132],
 [datetime.datetime(2023, 10, 27, 23, 11, 20), 158, 158],
 [datetime.datetime(2023, 10, 27, 23, 11, 44), 164, 164],
 [datetime.datetime(2023, 10, 27, 23, 12, 20), 173, 173],
 [datetime.datetime(2023, 10, 27, 23, 14, 36), 207, 207],
 [datetime.datetime(2023, 10, 27, 23, 17, 1), 243, 243],
 [datetime.datetime(2023, 10, 27, 23, 18, 49), 270, 270],
 [datetime.datetime(2023, 10, 27, 23, 20, 33), 296, 296],
 [datetime.datetime(2023, 10, 27, 23, 21, 21), 308, 308],
 [datetime.datetime(2023, 10, 27, 23, 21, 33), 311, 311],
 [datetime.datetime(2023, 10, 27, 23, 22, 17), 322, 322],
 [datetime.datetime(2023, 10, 27, 23, 22, 45), 329, 329],
 [datetime.datetime(2023, 10,

In [20]:
cursor.execute(f"""SELECT row_index
                        FROM (SELECT rul_fl, rul_pb, rul_ph, input_time, ROW_NUMBER() OVER (ORDER BY input_time) AS row_index
                        FROM rul_1) AS temp
                        WHERE (rul_fl < 10) or (rul_pb < 10) or (rul_ph < 10);""")
existing_user = cursor.fetchall()


In [21]:
line_lis = [val[0] for val in existing_user]
line_lis

[287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348]

In [17]:
line_lis = []

In [29]:
cursor.execute(f"""SELECT distinct DATE_FORMAT(input_time, '%H:%i:%s'), ACTUALROTATIONANGLE, FIXTURETILTANGLE,
                        ETCHBEAMCURRENT,IONGAUGEPRESSURE,
                        ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
                        ACTUALSTEPDURATION, ETCHSOURCEUSAGE,
                        FLOWCOOLFLOWRATE,FLOWCOOLPRESSURE
                    FROM input_data_1;""")
existing_user = cursor.fetchall()

In [34]:
existing_user[0]

('11:05:11',
 -0.1224370708389037,
 2.027847975076401,
 -0.9595093781327426,
 -0.0794283467426016,
 -1.751661148558742,
 -2.783162253978537,
 4.03778468798202,
 2.3953704185454097,
 -2.842906518897486,
 -2.3328068957350747)

In [39]:
for i in range(1,11):
    print(i)

1
2
3
4
5
6
7
8
9
10


In [38]:
for each in existing_user:
    if len(each) != 11:
        print(each)

# len(existing_user[0])

In [31]:
colnames = cursor.description  # 변수정보
cols = [[i, colnames[i][0], colnames[i+1][0]] for i in range(1, len(colnames), 2)]  # 변수명
    

In [33]:
existing_user[0],existing_user[1]

(('11:05:11',
  -0.1224370708389037,
  2.027847975076401,
  -0.9595093781327426,
  -0.0794283467426016,
  -1.751661148558742,
  -2.783162253978537,
  4.03778468798202,
  2.3953704185454097,
  -2.842906518897486,
  -2.3328068957350747),
 ('11:05:16',
  -0.1224370708389037,
  2.027847975076401,
  -0.9596512376809438,
  -0.0794187434958652,
  -1.7439048724585284,
  -2.7808841494321865,
  4.03778468798202,
  2.3953704185454097,
  -2.842906518897486,
  -2.3328068957350747))

In [40]:
# 각 val의 길이를 출력하여 확인
for val in existing_user:
    print(len(val))

# i의 최대 값이 val의 길이보다 작은지 확인
for val in existing_user:
    if i >= len(val):
        print(f"Index {i} is out of range for val with length {len(val)}")


11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
1

In [47]:
# i의 최대 값이 val의 길이보다 작은지 확인
for val in existing_user:
    if 11 >= len(val):
        print(f"Index 10 is out of range for val with length {len(val)}")


Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with length 11
Index 10 is out of range for val with le

In [42]:
print(f"i value: {i}")
for val in existing_user:
    print(f"Tuple length: {len(val)}")

i value: 10
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tuple length: 11
Tu

In [43]:
reference_length = len(existing_user[0])
for val in existing_user:
    if len(val) != reference_length:
        print(f"Tuple with different length found: {len(val)}")


In [45]:
existing_user

[('11:05:11',
  -0.1224370708389037,
  2.027847975076401,
  -0.9595093781327426,
  -0.0794283467426016,
  -1.751661148558742,
  -2.783162253978537,
  4.03778468798202,
  2.3953704185454097,
  -2.842906518897486,
  -2.3328068957350747),
 ('11:05:16',
  -0.1224370708389037,
  2.027847975076401,
  -0.9596512376809438,
  -0.0794187434958652,
  -1.7439048724585284,
  -2.7808841494321865,
  4.03778468798202,
  2.3953704185454097,
  -2.842906518897486,
  -2.3328068957350747),
 ('11:05:20',
  -0.1224370708389037,
  2.027847975076401,
  -0.960005916785792,
  -0.079408642268514,
  -1.7439048724585284,
  -2.7808841494321865,
  4.03778468798202,
  2.3953704185454097,
  -2.841849516088073,
  -2.3328068957350747),
 ('11:05:24',
  -0.1224370708389037,
  2.027847975076401,
  -0.9592965888104408,
  -0.0794198349017626,
  -1.7471575247996358,
  -2.7808841494321865,
  4.03778468798202,
  2.3953704185454097,
  -2.841849516088073,
  -2.3328068957350747),
 ('11:05:28',
  -0.1224370708389037,
  2.02784797507

In [44]:
if not existing_user:
    print("existing_user is empty!")
    return


SyntaxError: 'return' outside function (2388791911.py, line 3)

In [48]:
def fetch_bar_lis_from_database(n=1):
    cursor.execute(f"""SELECT distinct DATE_FORMAT(input_time, '%H:%i:%s'), ACTUALROTATIONANGLE, FIXTURETILTANGLE,
                        ETCHBEAMCURRENT,IONGAUGEPRESSURE,
                        ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
                        ACTUALSTEPDURATION, ETCHSOURCEUSAGE,
                        FLOWCOOLFLOWRATE,FLOWCOOLPRESSURE
                    FROM input_data_{n};""")
    existing_user = cursor.fetchall()
    
    colnames = cursor.description  # 변수정보
    cols = [[i, colnames[i][0], colnames[i+1][0]] for i in range(1, len(colnames), 2)]  # 변수명
    
    alram_dic = {}
    for i in range(1, len(existing_user[0])):
        key = 'alram{}'.format(i)
        alram_dic[key] = [{'time': val[0], 'col': val[i]} for val in existing_user]

        try:
            cursor.execute(f"""SELECT row_index
                            FROM (SELECT rul_fl, rul_pb, rul_ph, input_time, ROW_NUMBER() OVER (ORDER BY input_time) AS row_index
                            FROM rul_1) AS temp
                            WHERE (rul_fl < 10) or (rul_pb < 10) or (rul_ph < 10);""")
            existing_user = cursor.fetchall()
            line_lis = [val[0] for val in existing_user]
        
            cursor.execute(f"""SELECT temp.*, (row_index - LAG(row_index) OVER (ORDER BY row_index)) as diff
                            FROM (SELECT multi_pred_fl, multi_pred_pb, multi_pred_ph, input_time, ROW_NUMBER() OVER (ORDER BY input_time) AS row_index
                                FROM multi_{n}) AS temp
                            WHERE (multi_pred_fl = 1) or (multi_pred_pb = 1) or (multi_pred_ph = 1);""")
            existing_user = cursor.fetchall()
        
            bar_lis = [[existing_user[0][3], existing_user[0][4]]]
            cnt = 0
            for i in range(1, len(existing_user)):
                if existing_user[i][-1] != 1:
                    bar_lis[len(bar_lis) - 1].append(existing_user[i - 1][4])
                    bar_lis.append([existing_user[i][3], existing_user[i][4]])
            bar_lis[-1].append(existing_user[-1][4])

        except:
            line_lis = None
            bar_lis = [[None, 0, 0],[None, 0, 0],[None, 0, 0],[None, 0, 0]]
    
        return bar_lis

In [51]:
fetch_bar_lis_from_database()

[[datetime.datetime(2023, 10, 27, 6, 44, 45), 4513, 4517]]

In [56]:
cursor.execute(f"""SELECT temp.*, (row_index - LAG(row_index) OVER (ORDER BY row_index)) as diff
                        FROM (SELECT multi_pred_fl, multi_pred_pb, multi_pred_ph, input_time, ROW_NUMBER() OVER (ORDER BY input_time) AS row_index
                            FROM multi_{1}) AS temp
                        WHERE (multi_pred_fl = 1) or (multi_pred_pb = 1) or (multi_pred_ph = 1);""")
existing_user = cursor.fetchall()
# existing_user[0][-1] = existing_user[0][-2]
existing_user

[(0, 0, 1, datetime.datetime(2023, 10, 27, 23, 1, 43), 14, None),
 (1, 0, 1, datetime.datetime(2023, 10, 27, 23, 3, 23), 39, 25),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 4, 19), 53, 14),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 4, 23), 54, 1),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 5, 47), 75, 21),
 (1, 0, 1, datetime.datetime(2023, 10, 27, 23, 7, 52), 106, 31),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 9, 36), 132, 26),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 11, 20), 158, 26),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 11, 44), 164, 6),
 (1, 1, 0, datetime.datetime(2023, 10, 27, 23, 12, 20), 173, 9),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 14, 36), 207, 34),
 (1, 0, 0, datetime.datetime(2023, 10, 27, 23, 17, 1), 243, 36),
 (0, 0, 1, datetime.datetime(2023, 10, 27, 23, 18, 49), 270, 27),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 20, 33), 296, 26),
 (0, 1, 0, datetime.datetime(2023, 10, 27, 23, 21, 21), 308, 12),
 (0, 1, 0, datetime.date

In [53]:
bar_lis = [[existing_user[-1][3], existing_user[-1][4]]]
bar_lis

[[datetime.datetime(2023, 10, 29, 15, 58, 2), 1890]]

In [58]:
for i in range(len(existing_user) - 2, -1, -1):
    if existing_user[i][-1] != 1:
    # if existing_user[i][-1] != 1 :
        bar_lis[len(bar_lis) - 1].append(existing_user[i + 1][4])
        bar_lis.append([existing_user[i][3], existing_user[i][4]])
bar_lis


[[datetime.datetime(2023, 10, 29, 15, 58, 2), 1890, 1890],
 [datetime.datetime(2023, 10, 29, 15, 57, 10), 1877, 1877],
 [datetime.datetime(2023, 10, 29, 15, 54, 54), 1843, 1843],
 [datetime.datetime(2023, 10, 29, 15, 54, 5), 1831, 1831],
 [datetime.datetime(2023, 10, 29, 15, 53, 9), 1817, 1817],
 [datetime.datetime(2023, 10, 29, 15, 52, 13), 1803, 1803],
 [datetime.datetime(2023, 10, 29, 15, 51, 45), 1796, 1796],
 [datetime.datetime(2023, 10, 29, 15, 51, 17), 1789, 1789],
 [datetime.datetime(2023, 10, 29, 15, 49, 45), 1766, 1766],
 [datetime.datetime(2023, 10, 29, 15, 48, 33), 1748, 1748],
 [datetime.datetime(2023, 10, 29, 15, 48, 13), 1743, 1743],
 [datetime.datetime(2023, 10, 29, 15, 46, 12), 1713, 1713],
 [datetime.datetime(2023, 10, 29, 15, 42, 28), 1657, 1657],
 [datetime.datetime(2023, 10, 29, 15, 39, 56), 1619, 1619],
 [datetime.datetime(2023, 10, 29, 15, 39, 52), 1618, 1618],
 [datetime.datetime(2023, 10, 29, 15, 39, 3), 1606, 1606],
 [datetime.datetime(2023, 10, 29, 15, 37, 23

In [71]:
bar_lis = []

# 마지막 열 값이 >1 인 데이터와 None 값을 무시
for idx, row in enumerate(existing_user[:-1]):
    if row[-1] is not None and row[-1] > 1:
        current_datetime = row[3]
        current_value = row[-2]
        next_value = existing_user[idx + 1][-2]
        bar_lis.append((current_datetime, current_value, next_value))

# 전체 순서 역으로 정렬
bar_lis = sorted(bar_lis, key=lambda x: x[0], reverse=True)

In [72]:
filtered_data

[(datetime.datetime(2023, 10, 29, 15, 57, 10), 1877, 1890),
 (datetime.datetime(2023, 10, 29, 15, 54, 54), 1843, 1877),
 (datetime.datetime(2023, 10, 29, 15, 54, 5), 1831, 1843),
 (datetime.datetime(2023, 10, 29, 15, 53, 9), 1817, 1831),
 (datetime.datetime(2023, 10, 29, 15, 52, 13), 1803, 1817),
 (datetime.datetime(2023, 10, 29, 15, 51, 45), 1796, 1803),
 (datetime.datetime(2023, 10, 29, 15, 51, 17), 1789, 1796),
 (datetime.datetime(2023, 10, 29, 15, 49, 45), 1766, 1789),
 (datetime.datetime(2023, 10, 29, 15, 48, 33), 1748, 1766),
 (datetime.datetime(2023, 10, 29, 15, 48, 13), 1743, 1748),
 (datetime.datetime(2023, 10, 29, 15, 46, 12), 1713, 1743),
 (datetime.datetime(2023, 10, 29, 15, 42, 28), 1657, 1713),
 (datetime.datetime(2023, 10, 29, 15, 39, 52), 1618, 1619),
 (datetime.datetime(2023, 10, 29, 15, 39, 3), 1606, 1618),
 (datetime.datetime(2023, 10, 29, 15, 37, 23), 1581, 1606),
 (datetime.datetime(2023, 10, 29, 15, 34, 23), 1536, 1581),
 (datetime.datetime(2023, 10, 29, 15, 33, 7